In [1]:
from pyspark.sql import SparkSession, Window, functions as fun

In [2]:
spark = SparkSession.\
    builder.\
    appName("spaaaaaark").\
    config("spark.jars", "postgresql-42.4.0.jar").master("local").\
    getOrCreate()

22/09/26 08:02:50 WARN Utils: Your hostname, fenixfire-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.0.85 instead (on interface wlp7s0)
22/09/26 08:02:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/26 08:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_reader = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/postgres")\
    .option("user", "postgres")\
    .option("password", "qwert5")\
    .option("driver", "org.postgresql.Driver")

In [4]:
actor = df_reader.option('dbtable', 'actor').load()
address = df_reader.option('dbtable', 'address').load()
category = df_reader.option('dbtable', 'category').load()
city = df_reader.option('dbtable', 'city').load()
country = df_reader.option('dbtable', 'country').load()
customer = df_reader.option('dbtable', 'customer').load()
film = df_reader.option('dbtable', 'film').load()
film_actor = df_reader.option('dbtable', 'film_actor').load()
film_category = df_reader.option('dbtable', 'film_category').load()
inventory = df_reader.option('dbtable', 'inventory').load()
language = df_reader.option('dbtable', 'language').load()
payment = df_reader.option('dbtable', 'payment').load()
rental = df_reader.option('dbtable', 'rental').load()
staff = df_reader.option('dbtable', 'staff').load()
store = df_reader.option('dbtable', 'store').load()

In [7]:
# task 1

category.join(film_category, on='category_id')\
    .groupBy('name')\
    .count()\
    .orderBy(fun.col('count').desc())\
    .show()


+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



In [9]:
# task 2

actor.join(film_actor, on='actor_id')\
    .join(film, on='film_id')\
    .groupBy('first_name', 'last_name')\
    .agg(fun.sum('rental_duration').alias('sum_rent'))\
    .orderBy(fun.col('sum_rent').desc())\
    .show(10)

+----------+---------+--------+
|first_name|last_name|sum_rent|
+----------+---------+--------+
|     SUSAN|    DAVIS|     242|
|      GINA|DEGENERES|     209|
|    WALTER|     TORN|     201|
|      MARY|   KEITEL|     192|
|   MATTHEW|   CARREY|     190|
|   GROUCHO|    DUNST|     183|
|    ANGELA|   HUDSON|     183|
|    SANDRA|   KILMER|     181|
|     HENRY|    BERRY|     180|
|       UMA|     WOOD|     179|
+----------+---------+--------+
only showing top 10 rows



In [16]:
# task 3

category.join(film_category, on='category_id')\
    .join(film, on='film_id')\
    .groupBy('name')\
    .agg(fun.sum('rental_rate').alias('film_cost'))\
    .orderBy(fun.col('film_cost').desc())\
    .show(1)

+------+---------+
|  name|film_cost|
+------+---------+
|Sports|   231.26|
+------+---------+
only showing top 1 row



In [19]:
# task 4

film.join(inventory, on='film_id', how='left')\
    .groupby('film_id', 'title')\
    .agg(fun.count('inventory_id').alias('number'))\
    .filter(fun.col('number') == 0)\
    .select(fun.col('title'))\
    .orderBy('title')\
    .show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



In [22]:
# task 5

temp_table = category.join(film_category, on='category_id')\
    .join(film, on='film_id')\
    .join(film_actor, on='film_id')\
    .join(actor, on='actor_id')\
    .where(fun.col('category_id') == '3')\
    .groupBy('first_name', 'last_name').count()

count_filter = temp_table.select(fun.col('count'))\
    .orderBy(fun.col('count').desc())\
    .distinct()\
    .limit(3)\
    .rdd.map(lambda row : row[0]).collect()

temp_table.filter(fun.col('count').isin(count_filter)).orderBy(fun.col('count').desc()).show()

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|     HELEN|   VOIGHT|    7|
|     SUSAN|    DAVIS|    6|
|     KEVIN|  GARLAND|    5|
|     RALPH|     CRUZ|    5|
|      MARY|    TANDY|    5|
|    WHOOPI|     HURT|    5|
+----------+---------+-----+



In [24]:
# task 6

city.join(address, on='city_id')\
    .join(customer, on='address_id')\
    .groupBy('city')\
    .agg(fun.sum(fun.when(customer.active == 1, 1).otherwise(0)).alias('active'),
        fun.sum(fun.when(customer.active == 0, 1).otherwise(0)).alias('inactive'))\
    .orderBy(fun.col('inactive').desc())\
    .show()

+------------------+------+--------+
|              city|active|inactive|
+------------------+------+--------+
|         Pingxiang|     0|       1|
|       Szkesfehrvr|     0|       1|
|  Charlotte Amalie|     0|       1|
|         Najafabad|     0|       1|
|           Wroclaw|     0|       1|
|            Ktahya|     0|       1|
|           Bat Yam|     0|       1|
|   Southend-on-Sea|     0|       1|
|            Amroha|     0|       1|
|            Kamyin|     0|       1|
|          Xiangfan|     0|       1|
|            Daxian|     0|       1|
|          Uluberia|     0|       1|
|     Coatzacoalcos|     0|       1|
|        Kumbakonam|     0|       1|
|A Corua (La Corua)|     1|       0|
|          Fengshan|     1|       0|
|          Chisinau|     1|       0|
|           Udaipur|     1|       0|
|              Linz|     1|       0|
+------------------+------+--------+
only showing top 20 rows



In [32]:
# task 7

table_1 = rental.join(customer, on='customer_id')\
    .join(address, on='address_id')\
    .join(city, on='city_id')\
    .where(fun.substring(fun.col("city"), 1, 1) == 'a')\
    .groupBy("city")\
    .agg(fun.sum(fun.col("return_date")-fun.col("rental_date")).alias("rental_sum"))\
    .select("city", "rental_sum")

table_2 = rental.join(customer, on='customer_id')\
    .join(address, on='address_id')\
    .join(city, on='city_id')\
    .where(fun.locate('-', fun.col("city")) > 0)\
    .groupBy("city")\
    .agg(fun.sum(fun.col("return_date")-fun.col("rental_date")).alias("rental_sum"))\
    .select("city", "rental_sum")

table_1.union(table_2).show()

+--------------------+--------------------+
|                city|          rental_sum|
+--------------------+--------------------+
|           al-Hawiya|INTERVAL '79 03:1...|
|            al-Qatif|INTERVAL '119 16:...|
|          al-Qadarif|INTERVAL '69 05:1...|
|              al-Ayn|INTERVAL '124 12:...|
|           al-Manama|INTERVAL '126 22:...|
|     Jastrzebie-Zdrj|INTERVAL '155 16:...|
|    Shubra al-Khayma|INTERVAL '179 05:...|
|   Uttarpara-Kotrung|INTERVAL '139 00:...|
|Donostia-San Seba...|INTERVAL '175 00:...|
|      Kirovo-Tepetsk|INTERVAL '144 05:...|
| Kamjanets-Podilskyi|INTERVAL '153 15:...|
|        Shahr-e Kord|INTERVAL '100 17:...|
|       Hubli-Dharwad|INTERVAL '76 00:2...|
|Augusta-Richmond ...|INTERVAL '127 04:...|
|        Effon-Alaiye|INTERVAL '153 10:...|
|           al-Hawiya|INTERVAL '79 03:1...|
|      Tel Aviv-Jaffa|INTERVAL '127 13:...|
|    Jalib al-Shuyukh|INTERVAL '134 06:...|
|           Lapu-Lapu|INTERVAL '173 08:...|
|     s-Hertogenbosch|INTERVAL '